<a href="https://www.kaggle.com/code/shag2003/knn-algorithms?scriptVersionId=171845556" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/phishing-urls/Preprocessed_data.csv


In [2]:
dataset = pd.read_csv('/kaggle/input/phishing-urls/Preprocessed_data.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [3]:
dataset.head()

,login_form,links_in_tags,iframe,popup_window,safe_anchor,onmouseover,right_clic,whois_registered_domain,domain_registration_length,domain_age,...,having_ip_address,no_of_dots,no_of_sensitive_words,out_of_position_tld,https_token,url_length,tinyURL,prefixSuffix,count_special_chars,status
0,0,80.000000,0,0,0.0,0,0,0,45,-1,...,1,3,0,1,-1,37,-1,-1,7,legitimate
1,0,100.000000,0,0,100.0,0,0,0,77,5767,...,1,1,0,1,-1,77,-1,-1,7,phishing
2,0,100.000000,0,0,100.0,0,0,0,14,4004,...,1,4,1,-1,1,126,-1,1,19,phishing
3,0,100.000000,0,0,62.5,0,0,0,62,-1,...,1,2,0,1,-1,18,-1,-1,5,legitimate
4,1,76.470588,0,0,0.0,0,0,0,224,8175,...,1,2,0,1,-1,55,-1,-1,10,legitimate


## Splitting the dataset into the Training set and Test set

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [5]:
print(X_train)

[[  0.          50.           0.         ...  -1.           1.
    7.        ]
 [  0.           0.           0.         ...  -1.          -1.
   12.        ]
 [  0.          88.23529412   0.         ...  -1.          -1.
   13.        ]
 ...
 [  0.           0.           0.         ...  -1.          -1.
   14.        ]
 [  0.         100.           0.         ...  -1.          -1.
    8.        ]
 [  0.           0.           0.         ...  -1.          -1.
   10.        ]]


In [6]:
print(y_train)

['legitimate' 'legitimate' 'legitimate' ... 'phishing' 'phishing'
 'phishing']


In [7]:
print(X_test)

[[  0.   75.    0.  ...  -1.   -1.    6. ]
 [  0.    0.    0.  ...  -1.   -1.   10. ]
 [  0.    0.    0.  ...  -1.   -1.    6. ]
 ...
 [  0.  100.    0.  ...  -1.   -1.    6. ]
 [  0.   87.5   0.  ...  -1.   -1.    6. ]
 [  0.  100.    0.  ...  -1.   -1.    7. ]]


In [8]:
print(y_test)

['legitimate' 'phishing' 'phishing' ... 'legitimate' 'legitimate'
 'legitimate']


# Feature Scaling

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test) #avoid data leakage

In [10]:
print(X_train)

[[-0.25749132 -0.04794781 -0.03897267 ... -0.37846822  2.3119104
  -0.50373445]
 [-0.25749132 -1.25161465 -0.03897267 ... -0.37846822 -0.43254272
   0.35707885]
 [-0.25749132  0.8725033  -0.03897267 ... -0.37846822 -0.43254272
   0.52924151]
 ...
 [-0.25749132 -1.25161465 -0.03897267 ... -0.37846822 -0.43254272
   0.70140417]
 [-0.25749132  1.15571903 -0.03897267 ... -0.37846822 -0.43254272
  -0.33157179]
 [-0.25749132 -1.25161465 -0.03897267 ... -0.37846822 -0.43254272
   0.01275353]]


In [11]:
print(X_test.dtype)

float64


## Training the kNN model on the Training set

In [12]:
from math import sqrt
class KNN():
  def __init__(self,k):
    self.k=k
    print(self.k)
  def fit(self,X_train,y_train):
    self.x_train=X_train
    self.y_train=y_train
  def calculate_euclidean(self,sample1,sample2):
    distance=0.0
    for i in range(len(sample1)):
      distance+=(sample1[i]-sample2[i])**2 #Euclidean Distance = sqrt(sum i to N (x1_i – x2_i)^2)
    return sqrt(distance)
  def nearest_neighbors(self,test_sample):
    distances=[]#calculate distances from a test sample to every sample in a training set
    for i in range(len(self.x_train)):
      distances.append((self.y_train[i],self.calculate_euclidean(self.x_train[i],test_sample)))
    distances.sort(key=lambda x:x[1])#sort in ascending order, based on a distance value
    neighbors=[]
    for i in range(self.k): #get first k samples
      neighbors.append(distances[i][0])
    return neighbors
  def predict(self,test_set):
    predictions=[]
    for test_sample in test_set:
      neighbors=self.nearest_neighbors(test_sample)
      labels=[sample for sample in neighbors]
      prediction=max(labels,key=labels.count)
      predictions.append(prediction)
    return predictions

In [13]:
model=KNN(5) #our model
model.fit(X_train,y_train)

5


In [14]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)#The default metric is minkowski, and with p=2 is equivalent to the standard Euclidean metric.
classifier.fit(X_train, y_train)

KNeighborsClassifier()

## Predicting the Test set results

In [15]:
y_pred = classifier.predict(X_test)

In [16]:
predictions=model.predict(X_test) # our model's predictions

## Making the Confusion Matrix to compare both models

In [17]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1328  102]
 [ 129 1299]]


0.9191742477256823

In [18]:
cm = confusion_matrix(y_test, predictions) # our model
print(cm)
accuracy_score(y_test, predictions)

[[1328  102]
 [ 129 1299]]


0.9191742477256823